In [1]:
import tools
import param
import cartesian
import racks
import calibration
import samples
import json
import importlib
import random as rnd
import time
import low_level_comm as llc

In [3]:
llc.listSerialPorts()

['COM3', 'COM6', 'COM10']

# Preparation

1. Manually install the rack
    - Installed at 4, 3
2. Add config file for the rack to the folder "config".
    - saved in /config/384wells_rack.ini
3. Adjust geometry section (just physically measure parameters
4. Set the wells number and geometry
5. Change the settings for the rack calibration section

## Create a 384 well rack object for a first time

In [6]:
# rack_type must match config file name
rack384 = racks.rack(rack_name='384wells_rack', rack_type='384wells_rack', x_slot=4, y_slot=3)

In [7]:
rack384.rack_data

{'name': '384wells_rack', 'n_x': 4, 'n_y': 3, 'type': '384wells_rack'}

## Calibrate the rack

In [23]:
# TODO: re-calibrate the rack without the plate

In [8]:
ar = cartesian.arnie(cartesian_port="COM10", docker_port="COM6")

In [9]:
ar.home()

In [10]:
stp = tools.stationary_touch_probe(ar)

In [14]:
tp = tools.mobile_touch_probe.getTool(ar)

In [15]:
x, y, z = calibration.calibrateStationaryProbe(tp, stp)

In [16]:
ar.move(z=300)

In [17]:
xr, yr, zr = calibration.calibrateRack(tp, rack384)

In [18]:
tp.returnTool()

In [20]:
ar.move(x=10, y=10)

In [21]:
rack384.rack_data

{'name': '384wells_rack',
 'n_x': 4,
 'n_y': 3,
 'type': '384wells_rack',
 'position': [690.94, 395.9, 506.2],
 'pos_stalagmyte': [94.69999999999999, 65.55, 490.75]}

# Creating a new plate type

1. Creating a settings file
2. Modifying the dependence of the height vs. the volume for the well.

## Initializing a new plate object

In [35]:
p = samples.plate('384_test', '384wells_plate')
rack384well = racks.rack(rack_name='384wells_rack', rack_type='384wells_rack')
p.place(rack384well)

In [36]:
p.plate_data

{'rack': <racks.rack at 0x22af1025e48>}

## Attempting to pipette

In [27]:
r50 = racks.rack(rack_name='50mL_rack_1', rack_type='50mL')

In [28]:
blue = samples.sample(sample_name='blue', sample_type='50mL_tube')
blue.place(r50, 2, 0)
blue.setVolume(50000)

In [29]:
p200 = tools.pipettor.getTool(robot=ar, tool_name='p200_tool')

In [30]:
xt, yt, zt = calibration.calibrateTool(p200, stp)

In [31]:
ar.move(z=300)

In [32]:
p200_tip_rack = racks.consumables(rack_name='p200_tips_1', rack_type='p200_tips')

In [45]:
waste_rack = racks.rack(rack_name='waste', rack_type='waste_rack')

In [33]:
xn, yn = p200_tip_rack.getNextConsumable()
p200.pickUpTip(p200_tip_rack, xn, yn, raise_z=400)

In [39]:
p200.moveLiquid(blue, p.getSample(0, 0), 100, blow_extra=True, raise_z=350)

In [40]:
p200.moveLiquid(p.getSample(0, 0), blue, 100, blow_extra=True, raise_z=350)

In [42]:
samples_list = [
    p.getSample(0, 0),
    p.getSample(1, 1),
    p.getSample(2, 2),
    p.getSample(3, 3),
    p.getSample(4, 4),
]
vol_list = [
    20, 20, 20, 20, 20,
]

In [43]:
p200.distributeLiquid(blue, samples_list, vol_list, raise_z=400, touch_wall=True)

In [44]:
ar.move(z=400)

In [46]:
p200.dropTipToWaste(waste_rack)

In [47]:
p200.returnTool()

In [48]:
ar.move(x=10, y=10)